In [47]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

# Load the inference data
df_inference = pd.read_csv('padel_shots_inference_dataset.csv')
print("Inference data shape:", df_inference.shape)

# Load the trained model
model = load_model('_results/ws_7_weight_1_5/padel_model_auc_0.8696.keras')

# Prepare the data for inference
X_inference = df_inference.values  # Exclude the 'label' column
scaler = StandardScaler()
X_inference_scaled = scaler.fit_transform(X_inference)
X_inference_scaled = X_inference_scaled.reshape((X_inference_scaled.shape[0], 1, X_inference_scaled.shape[1]))

# Run inference
predictions_prob = model.predict(X_inference_scaled)
predictions = (predictions_prob > 0.6).astype(int).flatten()

# Calculate the number of original frames
# Assuming the inference dataset uses a 7-frame window and each row represents the middle frame
num_original_frames = len(predictions) + 6

# Create a list to store frame-wise predictions
frame_predictions = [0] * num_original_frames

# Map window predictions back to original frames
for i, pred in enumerate(predictions):
    frame_index = i + 3  # +3 because the prediction is for the middle frame of the 7-frame window
    frame_predictions[frame_index] = pred

# Fill in the first three and last three frames
frame_predictions[0] = frame_predictions[3]
frame_predictions[1] = frame_predictions[3]
frame_predictions[2] = frame_predictions[3]
frame_predictions[-3] = frame_predictions[-4]
frame_predictions[-2] = frame_predictions[-4]
frame_predictions[-1] = frame_predictions[-4]

print("Number of frames:", len(frame_predictions))
print("First 10 predictions:", frame_predictions[:10])
print("Last 10 predictions:", frame_predictions[-10:])

# If you want to save the predictions to a file
np.savetxt('frame_predictions.txt', frame_predictions, fmt='%d')
print("Predictions saved to 'frame_predictions.txt'")

Inference data shape: (1454, 266)
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Number of frames: 1460
First 10 predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Last 10 predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Predictions saved to 'frame_predictions.txt'
